<a href="https://colab.research.google.com/github/SylwiaNowakowska/LLM_Fine_Tuning_Molecular_Properties/blob/main/02_ChemBERTa2_Embedding_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChemBERTa-2 embeddings extraction
Notebook for embeddings extractions from last hidden layer.

## Environment setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --pre deepchem
import deepchem
deepchem.__version__

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 838.8/838.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 57.7 MB/s eta 0:00:00


'2.7.2.dev'

In [ ]:
!pip install transformers
!pip install simpletransformers
!cd ..

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 698.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 112.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 27.3 MB/s eta 0:00:0

In [ ]:
import sys
!test -d bertviz_repo && echo "FYI: bertviz_repo directory already exists, to pull latest version uncomment this line: !rm -r bertviz_repo"
# !rm -r bertviz_repo # Uncomment if you need a clean pull from repo
!test -d bertviz_repo || git clone https://github.com/jessevig/bertviz bertviz_repo
if not 'bertviz_repo' in sys.path:
  sys.path += ['bertviz_repo']
!pip install regex

Cloning into 'bertviz_repo'...
remote: Enumerating objects: 1633, done.
remote: Counting objects: 100% (329/329), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 1633 (delta 231), reused 225 (delta 211), pack-reused 1304
Receiving objects: 100% (1633/1633), 198.37 MiB | 16.68 MiB/s, done.
Resolving deltas: 100% (1073/1073), done.


In [ ]:
!git clone https://github.com/seyonechithrananda/bert-loves-chemistry.git

Cloning into 'bert-loves-chemistry'...
remote: Enumerating objects: 1566, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 1566 (delta 96), reused 91 (delta 91), pack-reused 1364
Receiving objects: 100% (1566/1566), 55.35 MiB | 15.78 MiB/s, done.
Resolving deltas: 100% (1000/1000), done.


## Imports and parameters setting

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import torch

from rdkit import Chem
from transformers import AutoModelForMaskedLM, AutoTokenizer, pipeline, RobertaModel, RobertaTokenizer
from typing import List

# import MoleculeNet loaders from DeepChem
from deepchem.molnet import load_bbbp, load_clearance, load_clintox, load_delaney, load_hiv, load_qm7, load_tox21

# import MoleculeNet dataloder from bert-loves-chemistry fork
%cd /content/bert-loves-chemistry
from chemberta.utils.molnet_dataloader import load_molnet_dataset, write_molnet_dataset_for_chemprop

In [ ]:
# enter the location of the model
project_name = "ChemBERTA_HIV"
output_path = '/content/drive/MyDrive/COLAB_NOTEBOOKS/LLM_Fine_Tuning_Molecular_Properties_output'
model_name = 'model_36'

model_folder = os.path.join(output_path, model_name)
evaluation_folder = os.path.join(output_path, model_name + '_evaluation')

embeddings_folder = os.path.join(output_path, model_name + '_embeddings')
if not os.path.exists(embeddings_folder):
    os.makedirs(embeddings_folder)

print(model_folder)

/content/drive/MyDrive/COLAB_NOTEBOOKS/LLM_Fine_Tuning_Molecular_Properties_output/model_36


## Loading predictions

In [ ]:
# load the dataframe with predictions
predictions_df_path = os.path.join(evaluation_folder, 'predictions_df.csv')
predictions_df = pd.read_csv(predictions_df_path, index_col=0)
predictions_df.head()

,text,labels,class_0_probability,class_1_probability,y_pred
0,CN=C1OC2(c3ccccc3)C3=C(OC(=NC)N2C)C(=O)OC3(c2c...,0,1.991554,-2.474177,0
1,c1ccc2c(c1)[nH]c1c3ccccc3sc21,0,1.659338,-2.097368,0
2,CN1C(=O)N(C)P(=O)(c2ccccc2)P1c1ccccc1,0,1.971669,-2.436386,0
3,O=C(c1ccccc1)N1OCCCc2ccccc21,0,1.928813,-2.344722,0
4,COC(=O)c1c2c(cc3nc4ccccc4nc13)OCO2,0,1.767895,-2.166682,0


## Pre-trained & fine-tined model: calculate the embeddings

In [ ]:
import torch
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from transformers import AutoTokenizer, AutoModel

# Load the tokenizers and models
tokenizer = AutoTokenizer.from_pretrained('DeepChem/ChemBERTa-77M-MLM')
model_before = AutoModel.from_pretrained('DeepChem/ChemBERTa-77M-MLM')
model_after = AutoModel.from_pretrained(model_folder)

In [ ]:
def calculate_embeddings(df, model, tokenizer, output_path=None):
    text_data = df['text'].tolist()
    labels = df['labels'].tolist()
    inputs = tokenizer(text_data, padding=True, truncation=True, return_tensors='pt', max_length=128)

    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state[:, 0, :].numpy()

    if output_path:
        np.save(output_path, embeddings)

    return embeddings

In [ ]:
id_string = 'test'
embeddings_before_path = os.path.join(embeddings_folder, id_string + '_embeddings_before.npy')
embeddings_after_path = os.path.join(embeddings_folder, id_string + '_embeddings_after.npy')

test_embeddings_before = calculate_embeddings(predictions_df, model_before, tokenizer, output_path=embeddings_before_path)
test_embeddings_after = calculate_embeddings(predictions_df, model_after, tokenizer, output_path=embeddings_after_path)

## Embeddings: dimensionality reduction

In [ ]:
test_embeddings_before = np.load(embeddings_before_path)
test_embeddings_after = np.load(embeddings_after_path)

test_embeddings_pca_before_path = os.path.join(embeddings_folder, 'test_embeddings_pca_before.npy')
test_embeddings_pca_after_path = os.path.join(embeddings_folder, 'test_embeddings_pca_after.npy')

In [ ]:
from sklearn.decomposition import PCA

# Perform PCA for dimensionality reduction
pca = PCA()
test_embeddings_pca_before = pca.fit_transform(test_embeddings_before)
np.save(test_embeddings_pca_before_path, test_embeddings_pca_before)
test_embeddings_pca_after = pca.fit_transform(test_embeddings_after)
np.save(test_embeddings_pca_after_path, test_embeddings_pca_after)